In [2]:
class Ensemble(object):
    def __init__(self, name, tau_rc, tau_ref, n_neurons, dims):
        self.name = name
        self.tau_rc = tau_rc
        self.tau_ref = tau_ref
        self.n_neurons = n_neurons
        self.dims = dims
        self.connections = {}
    def add(self, decoder, target_name, transform, synapse):
        if synapse not in self.connections:
            self.connections[synapse] = set()
        self.connections[synapse].add(target_name)


class Reader(object):
    def __init__(self, filename):
        self.ensembles = {}
        self.read(open(filename))

    def total_neurons(self):
        return sum(ens.n_neurons for ens in self.ensembles.values())

    def read(self, f):
        ens = None
        for line in f.readlines():
            if line.startswith(' '):
                line = line.strip()
                decoder, target = line.split(', ', 1)
                assert target[0] == '['
                assert target[-1] == ']'
                target = target[1:-2]
                if len(target) > 0:
                    targets = target.split('),')
                    for t in targets:
                        t = t[1:]
                        name, t = t.split(', ', 1)
                        transform, synapse = t.rsplit(', ', 1)
                        while name[-1] == '#':
                            name = name[:-1]
                        ens.add(decoder, name, transform, synapse)
            else:
                data = line.strip().split(', ')
                name, tau_rc, tau_ref, n_neurons, dims = data
                ens = Ensemble(name=name,
                               tau_rc=float(tau_rc),
                               tau_ref=float(tau_ref),
                               n_neurons=int(n_neurons),
                               dims=int(dims))
                assert name not in self.ensembles
                self.ensembles[name] = ens


In [3]:
r = Reader('spaun-v5.txt')

In [4]:

r.total_neurons()

2742962

In [5]:
def fan_out(reader):
    total_out = 0
    for ens in reader.ensembles.values():
        ens.fan_out = 0
        for syn, targets in ens.connections.items():
            ens.fan_out += sum(reader.ensembles[n].n_neurons for n in targets)
        total_out += ens.n_neurons*ens.fan_out
    return total_out

fan_out(r)

70378003180

In [6]:
def connections(reader):
    conns = []
    
    for pre_name, ens in reader.ensembles.items():
        for syn, targets in ens.connections.items():
            for post_name in targets:
                n_pre = ens.n_neurons
                n_post = reader.ensembles[post_name].n_neurons
                n_weights = n_pre*n_post
                c = (pre_name, post_name, syn, n_pre, n_post)
                conns.append(c)
    return conns

conns = connections(r)

In [7]:
print sum(c[3]*c[4] for c in conns)

70378003180


In [8]:
print len(conns)

2467349


In [9]:
def save_conns(filename, conns):
    f = open(filename, 'w')
    for c in conns:
        f.write('%s, %s, %s, %d, %d\n' % c)
    f.close()
save_conns('spaun-v5-conns.txt', conns)

KeyboardInterrupt: 